In [3]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel

In [4]:
class ERPDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

In [5]:
class ERPIntentClassifier(nn.Module):
    def __init__(self, n_classes):
        super().__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.drop = nn.Dropout(0.3)
        self.fc = nn.Linear(768, n_classes)
        
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        pooled_output = outputs[1]
        output = self.drop(pooled_output)
        return self.fc(output)


In [6]:
intent_examples = {
    'check_invoice': [
    "Show my pending invoices",
    "Check invoice status",
    "Find invoice details",
    "What are my unpaid invoices?",
    "Show me the details of my last invoice",
    "Can you check if I have any open invoices?",
    "Give me the status of my latest invoice",
    "Check if there's any pending payment on my invoices",
    "I need to see my outstanding invoices",
    "What is the balance on my invoice?",
    "Is there any due amount in my invoices?",
    "Can I see my invoice summary?",
    "Display my unpaid invoices",
    "Tell me the amount due on my invoices",
    "Can I view my payment history?",
    "Show me a list of all my invoices",
    "I want to know the due date of my invoices",
    "Check for any overdue invoices",
    "Can you provide me with the invoice details?",
    "I need an update on my invoices",
    "What’s the current status of my invoices?",
    "Have all my invoices been paid?",
    "How much do I owe on my invoices?",
    "Give me a breakdown of my pending invoices",
    "Can you show me the most recent invoice?",
    "What’s the remaining balance on my invoices?",
    "Show me the status of the invoices from last month",
    "Are there any unpaid invoices I should be aware of?",
    "Can I see an invoice for last quarter?",
    "Have I paid all my invoices this year?",
    "Tell me when my next invoice is due",
    "Find me the invoice from December",
    "What was the total amount of my last invoice?",
    "Do I have any outstanding payments?",
    "Can you show me any invoices that are past due?",
    "Check if I've received the invoice for my last order",
    "Give me an overview of all invoices for this year",
    "Show all invoices in the last 60 days",
    "What’s the payment status of my recent invoices?",
    "Can I have a copy of my last invoice?",
    "Give me the amount due for this particular invoice",
    "When was my last invoice created?",
    "Show me all invoices that are marked as unpaid",
    "Do I have any invoices from last month?",
    "Can you filter invoices that are not paid yet?",
    "What’s the total unpaid amount on all invoices?",
    "Show me invoices for this year",
    "Find the invoice from last week",
    "Can you list my most recent invoices?",
    "Give me the invoice details for my last payment",
    "Can I check the payment status for my current invoices?",
    "Is there a due date for my last invoice?",
    "What’s the payment status of the last few invoices?",
    "Can I get an invoice for the past month?",
    "Please show me my unpaid invoices for this quarter",
    "Can you check the payment status for all outstanding invoices?",
    "How many invoices are still due?",
    "Are there invoices from last year that haven't been paid?",
    "Please show my overdue invoices",
    "I want to check which invoices are still unpaid",
    "Have I received any new invoices recently?",
    "Show me details of invoices for January",
    "Give me an invoice summary for this month",
    "What’s the total value of my unpaid invoices?",
    "Please give me the invoice number for the latest payment",
    "I need the details for my most recent invoice",
    "How many invoices do I have to pay?",
    "Can you show me if there are any partial payments on invoices?",
    "Are there any invoices pending that I haven't paid?",
    "Do I owe anything for my recent orders?",
    "Can you list all invoices from the past six months?",
    "What was the due amount on my last invoice?",
    "Can you send me an updated invoice?",
    "Check if any of my invoices are overdue",
    "Find the invoice for my last transaction",
    "Can you tell me the due dates for all pending invoices?",
    "What invoices are pending for payment this month?",
    "List invoices that are overdue for more than 30 days",
    "Show me details for the invoice created last Friday",
    "Check if the invoice for my order number 123 is paid",
    "Show me my invoice summary with unpaid amounts",
    "Find an invoice for the amount of $500",
    "Tell me the status of invoices from my last two orders",
    "Are there any invoices I need to pay immediately?",
    "Show me the total unpaid balance of all invoices",
    "Give me a list of invoices that are due soon",
    "Check if I have any pending invoices from last month",
    "Provide a summary of my paid and unpaid invoices",
    "Can I see the invoices for my last purchase?",
    "What are the charges listed on my invoice?",
    "Is there any remaining balance on my account?",
    "Do you have my invoices from last year?",
    "Can I get a PDF copy of my invoice?",
    "Show all the invoices with a due date next week",
    "Are there any invoices in my account that I missed?",
    "Provide a list of all the invoices with pending payments",
    "Check if I've received an invoice for my recent service",
    "Give me a summary of invoices for the last 3 months",
    "Do I have any invoices with an outstanding balance?"
],
    'check_stock': [
    "What's the current inventory",
    "Check stock levels",
    "Show product quantity",
    "How many units of this product are available?",
    "Check if we have any stock left",
    "Is this item in stock?",
    "What is the current stock of product X?",
    "How many of these items do we have?",
    "Check availability of the product",
    "Do you have enough stock of this item?",
    "Show me how many items are left in the warehouse",
    "How much stock do we have of this product?",
    "Can you check if this product is available?",
    "Give me the stock count for this product",
    "What’s the available quantity for item Y?",
    "Can you check the current stock for item X?",
    "How many units are in stock for product Z?",
    "Tell me how many products are left in inventory",
    "Do we have any stock left for this item?",
    "Can you provide the stock level of item X?",
    "Is product Y currently available?",
    "How many units do we have in stock for product A?",
    "Can you find out if the product is available?",
    "How many of this item is left in stock?",
    "Please show me the stock levels for all items",
    "Are there any items running low on stock?",
    "What is the stock level of product B?",
    "Can you give me the stock level for this product?",
    "How many of product Z are in inventory?",
    "Show me the current stock for product X",
    "Please provide the stock quantity for this product",
    "What’s the current stock of the product I need?",
    "How many units of this product are in stock?",
    "Tell me the current stock for product Y",
    "Do we have enough of this item in stock?",
    "Check the stock levels for item B",
    "How many products are available right now?",
    "Are there any shortages in stock?",
    "Check if we have enough inventory for product X",
    "Give me the inventory levels for all products",
    "Tell me the stock for all items in the warehouse",
    "Is the product available for immediate purchase?",
    "Can you show me the stock levels of product C?",
    "How many items are available for product X?",
    "Please check if we’re out of stock on this product",
    "How much inventory do we have for item A?",
    "What’s the availability of this product?",
    "Can you show me the stock levels for the next quarter?",
    "Tell me how much of product Y is available",
    "Can you check if there's more inventory coming?",
    "Find the stock for product Z in the warehouse",
    "What’s the total quantity available for item A?",
    "How many units are available for product A?",
    "Check availability for the product I ordered",
    "Can you tell me how many products are available?",
    "Do we have enough stock to fulfill all orders?",
    "Can I place an order if stock is low?",
    "What products are out of stock right now?",
    "Can you give me the current inventory for all products?",
    "What’s the availability of the product I want to buy?",
    "How many of this product are in the warehouse?",
    "Do you have this product in stock?",
    "Can I check stock levels for all my orders?",
    "Please provide me with stock levels for next week",
    "Is the stock of this item enough to meet demand?",
    "Check if the item I need is out of stock",
    "Can you tell me if the product is available right now?",
    "How much stock is available for immediate shipment?",
    "Tell me if there’s more inventory arriving soon",
    "Can you give me an update on product availability?",
    "Check if this product is back in stock",
    "How many units do we have in stock for this product?",
    "What is the stock level for product A today?",
    "How many units of this product are available for order?",
    "Can you provide the stock count for this item today?",
    "How much inventory is available in the warehouse?",
    "What’s the stock status of product B right now?",
    "Is this item currently in stock in the store?",
    "Tell me the inventory count for product X",
    "How many units of product Y are available for sale?",
    "What’s the current stock of product A in the warehouse?",
    "Are there any low stock products?",
    "What are the stock levels for the top-selling products?",
    "Do we have enough stock to restock the shelves?",
    "How many of this item are currently in stock?",
    "Can you provide stock information for all the items?",
    "Check how much stock we have for this product",
    "Is this product available in large quantities?",
    "How many units of item X are left?",
    "What’s the stock availability for the coming week?",
    "Can you show me the stock of product B in the store?",
    "Find the current stock levels for the products",
    "Please tell me if any items are out of stock",
    "What is the stock count for product C?",
    "Check if there is sufficient inventory for product X"
]
}

In [7]:
def prepare_data():
    texts = []
    labels = []
    label_to_id = {}
    
    for idx, (intent, examples) in enumerate(intent_examples.items()):
        label_to_id[intent] = idx
        texts.extend(examples)
        labels.extend([idx] * len(examples))
    
    return texts, labels, label_to_id

In [8]:

def train_model():
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    texts, labels, label_to_id = prepare_data()
    
    dataset = ERPDataset(texts, labels, tokenizer)
    dataloader = DataLoader(dataset, batch_size=16)
    
    model = ERPIntentClassifier(len(label_to_id))
    optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
    criterion = nn.CrossEntropyLoss()
    
    model.train()
    for epoch in range(3):
        for batch in dataloader:
            optimizer.zero_grad()
            input_ids = batch['input_ids']
            attention_mask = batch['attention_mask']
            labels = batch['label']
            
            outputs = model(input_ids, attention_mask)
            loss = criterion(outputs, labels)
            
            loss.backward()
            optimizer.step()
    
    return model, tokenizer, label_to_id

In [9]:
def predict_intent(text, model, tokenizer, label_to_id):
    model.eval()
    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=128,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
    
    with torch.no_grad():
        outputs = model(encoding['input_ids'], encoding['attention_mask'])
        _, predicted = torch.max(outputs, 1)
    
    id_to_label = {v: k for k, v in label_to_id.items()}
    return id_to_label[predicted.item()]

In [10]:
model, tokenizer, label_to_id = train_model()

In [13]:
test_queries = [
        "Why cannot I download my invoice?",
        "What's the inventory?",
]

In [14]:
for query in test_queries:
        intent = predict_intent(query, model, tokenizer, label_to_id)
        print(f"Query: {query}\nPredicted Intent: {intent}\n")

Query: Why cannot I download my invoice?
Predicted Intent: check_invoice

Query: What's the inventory?
Predicted Intent: check_stock

